##런타임 유형 변경 -> GPU로

In [2]:
##bertopic 설치

!pip install bertopic[visualization]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB

In [3]:
import requests
import time
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re
import pickle
from bertopic import BERTopic

In [4]:
#불용어 불러오기

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
#nba 리뷰 데이터 불러오기

with open('/content/drive/MyDrive/NBA_review.pickle', 'rb') as f:
    df = pickle.load(f)

In [6]:
def remove_special_characters(text):
    # 알파벳과 숫자만을 남기고 나머지 특수문자 제거
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [7]:
# 특수 문자 제거
df['clean_review'] = df['review'].apply(lambda x: remove_special_characters(x))

# 길이가 2이하인 단어는 제거 (길이가 짧은 단어 제거)
df['clean_review'] = df['clean_review'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# 전체 단어에 대한 소문자 변환
df['clean_review'] = df['clean_review'].apply(lambda x: x.lower())

In [8]:
# NLTK로부터 불용어를 받아온다.
stop_words = stopwords.words('english')
tokenized_doc = df['clean_review'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

In [9]:
# 토큰이 1개거나 0개인 리뷰 제거 + 너무 많은 단어 있는것 제거

tokenized_doc = [i for i in tokenized_doc if len(i)>1]
tokenized_doc = [i for i in tokenized_doc if len(i)< 1000]
len(tokenized_doc)

7689

In [10]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(tokenized_doc)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

##NBA 토픽모델링 모델 불러오기

In [17]:
loaded_model = BERTopic.load("/content/drive/MyDrive/nba_model")

In [18]:
loaded_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2764,-1_game_play_even_dont,"[game, play, even, dont, like, money, next, st...",NaN
1,0,186,0_player_make_basketball_players,"[player, make, basketball, players, shot, team...",NaN
2,1,148,1_jogo_para_lang_mais,"[jogo, para, lang, mais, cheeks, juego, tudo, ...",NaN
3,2,137,2_controller_keyboard_settings_button,"[controller, keyboard, settings, button, contr...",NaN
4,3,132,3_good_game_teehee_hehe,"[good, game, teehee, hehe, haha, funny, says, ...",NaN
...,...,...,...,...,...
127,126,11,126_bench_court_skip_watch,"[bench, court, skip, watch, slow, reasom, ange...",NaN
128,127,11,127_garbage_soulless_pray_absolutely,"[garbage, soulless, pray, absolutely, ever, st...",NaN
129,128,11,128_look_disgraceful_inflated_purchasing,"[look, disgraceful, inflated, purchasing, grap...",NaN
130,129,11,129_crash_restart_freezes_cashoh,"[crash, restart, freezes, cashoh, refundable, ...",NaN


In [19]:
loaded_model.visualize_hierarchy()

In [20]:
num_topics = [[24, 114, 2],[33, 35],[5,29,113],[38,92],[75,127],[120,8,58],[87,90,31,91],[40, 44],[103, 55, 97],[84, 18],[100, 77],[93, 73],[56, 102],[108, 12],[119,49,109],[105,78,121,43],
[48,60,26,112],[32,30,25,47,66,10,129],[72,117,116],[21,9,23,123],[110,46],[62,89,42],[17,128,0,45,85,130],[86,7,74],[107,13,37,68],[67,83,65,101,70],[20, 126],[96, 57, 39],[106, 34, 98],
[122, 28],[118, 54, 104],[63, 16, 51],[41, 94],[36, 69, 15],[80, 124],[22, 14, 95, 61],[71, 115],[88, 59, 99, 79],[6, 50],[64, 4],[27, 81],[19, 111],[82, 52, 125],[53, 3, 11, 76]]

In [21]:
##토픽 합치기

loaded_model.merge_topics(detokenized_doc, num_topics)

In [22]:
loaded_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2764,-1_game_play_even_dont,"[game, play, even, dont, like, money, next, st...",[game fucking sucks tried playing game mycaree...
1,0,326,0_player_make_game_like,"[player, make, game, like, play, basketball, e...",[tldr game trash purchase bunch cheaters terri...
2,1,314,1_crashing_crashes_keeps_always,"[crashing, crashes, keeps, always, crash, myca...","[game keeps crashing, game keeps crashing, gam..."
3,2,271,2_good_game_really_great,"[good, game, really, great, pretty, rlly, rehe...","[good game, good game, good game]"
4,3,216,3_2k23_2k24_better_2k22,"[2k23, 2k24, better, 2k22, worse, 2k21, next, ...","[like 2k23, game 2k23, 2k23 next 2k24]"
5,4,205,4_controller_keyboard_start_button,"[controller, keyboard, start, button, settings...",[dont buggy game cant even press start button ...
6,5,187,5_story_mycareer_mode_career,"[story, mycareer, mode, career, storyline, lin...","[story mycareer, mycareer story, mycareer story]"
7,6,177,6_great_graphics_amazing_recommend,"[great, graphics, amazing, recommend, good, ga...","[great game, great game good graphics need nex..."
8,7,161,7_version_console_port_next,"[version, console, port, next, consoles, xbox,...","[next version, next console, like version cons..."
9,8,155,8_cheaters_cheats_cheater_banned,"[cheaters, cheats, cheater, banned, cheat, ful...","[full cheaters, full cheaters, full cheaters]"


In [24]:
##토픽 별 자주 등장하는 단어

loaded_model.visualize_barchart()

In [23]:
##미래의 리뷰 토픽 분류

reviews = "too many hacks"

topics, probs = loaded_model.transform(reviews)
print('예측한 토픽 번호 :', topics)

예측한 토픽 번호 : [28]
